In [ ]:
'''
Analyze the accelerometer data collected from a slipper walking on textured ground surfaces
Created on 2024.03.16 (shaoyitian@hit.edu.cn)
'''

# %matplotlib notebook
# %matplotlib notebook

# Import packages
import time
from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import mlab
import pandas as pd
import scipy.io as scio
from scipy import signal
# import seaborn as sns
import IPython.display as ipd
from IPython.core.display import HTML

from librosa.feature import mfcc

''' Figure format'''
plt.rc('font', size=16, family='Verdana') # 'Tahoma', 'DejaVu Sans', 'Verdana'"
plt.rc('axes', edgecolor='k', linewidth=0.75, labelcolor='k')
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':True, 'top':True})
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['errorbar.capsize'] = 4

''' Define Color Here '''
pltBlue = (32/255,120/255,180/255)
pltRed = (180/255,32/255,32/255)

''' Suppress warnings '''
import warnings
warnings.filterwarnings('ignore')

''' 
General Settings
'''
AudioFs = 44100 # (Hz) Sampling frequency of the audio measurements

'''
General Functions
'''

def decodeData(fileName, decodeFormat, frontCode='', rearCode='', isString=False):
    segStr = re.findall(frontCode+decodeFormat+rearCode, fileName)
    if segStr:
        decoded = re.findall(decodeFormat, segStr[0])[0]      
        if isString:
            return decoded  
        return float(decoded)
    return None

def lowpassSmooth(datain, cutFreqRatio=0.05, order=8):
    b, a = signal.butter(order, 2 * cutFreqRatio, btype='low')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def highpassFilter(datain, cutFreqRatio=0.01, order=8):
    b, a = signal.butter(order, 2 * cutFreqRatio, btype='high')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def movAvgSmooth(datain, winLen=100):
    dataout = np.convolve(datain, np.ones(winLen)/winLen, mode='same')
    return dataout

def segmentByEnergy(datain, segIntervalSamp, disp=False, threshold=0.1):
    ''' This function segment the accelerometer data based on detecting the energy surge of the input signal '''
    # 'datain' - a data vector used to determine the segmentation
    # 'segIntervalSamp' - the number of samples for each detection window (sliding window length)
    # 'disp' - display the segmentation result if True
    # 'threshold' - the threshold to determine whether the energy has increased or decreased significantly
    
    dataEnergy = abs(datain-np.mean(datain)) # Energy of the acceleration signals
    
    smoothSig = movAvgSmooth(dataEnergy, segIntervalSamp) # Identify the energy fluctuation with moving-average smoothing
        
    samp = np.arange(len(datain)-1) # Shorten by 1 samples for indexing after computing the 1st difference
    energyDiff = np.diff(smoothSig)
    energyIncreaseInd = samp[energyDiff > threshold] # Index of samples when the energy increased significantly
    energyDecreaseInd = samp[energyDiff < -threshold] # Index of samples when the energy decreased significantly
    
    tmp = energyIncreaseInd[:-1]
    energySurgeUpInd = tmp[np.diff(energyIncreaseInd) > (segIntervalSamp * 0.5)] # Enforce a minimal duration bewteen two consecutive energy increasing intervals
    
    tmp = energyDecreaseInd[:-1]
    energySurgeDownInd = tmp[np.diff(energyDecreaseInd) > (segIntervalSamp * 0.5)] # Enforce a minimal duration bewteen two consecutive energy decreasing intervals

    if disp:
        _,ax0 = plt.subplots(1,1,figsize=(14, 6),dpi=72); 
        ax0.plot(samp, datain[:-1], color='tab:grey'); 
        axb = ax0.twinx() 
        axb.plot(samp, smoothSig[:-1], color='tab:green') # Plot the smoothed signals
        axb.tick_params(axis='y', labelcolor='tab:green')
        ax0.plot(energySurgeUpInd, np.zeros(energySurgeUpInd.shape), '.r')
        ax0.plot(energySurgeDownInd, np.zeros(energySurgeDownInd.shape), '.b')
        plt.show();
        
    return energySurgeUpInd, energySurgeDownInd


In [ ]:
'''
Import and preprocessing of accelerometer measurements
'''
DataPath = './data2024.03'

''' Import, metadata extraction, and segmentation '''

segIntervalSamp = 600 # (samples) The window length used for onset detection and segmentation

selectAxis = 2

walkDF = [] # Dataframe of the walking measurements
for root, directories, files in walk(DataPath):
    for fileName in files:
        if root is not None:
            metaDataStings = root.split("\\") # Acquire metadata from the root string of the data
            majorType = metaDataStings[1] # The parent folder is named by the major surface type
            subType = metaDataStings[2] # The child folder is named by the subcategory type

            dataName = decodeData(fileName, '\w+', frontCode='', rearCode='.csv', isString=True)
            accFs = decodeData(fileName, '[\d+\.]*\d+', frontCode='Fs', rearCode='Hz.csv')

            if (dataName is not None) and (accFs is not None):
                readData = pd.read_csv(ospa.join(root, fileName), header = None)
                if readData is not None:
                    accData = readData.to_numpy() # Convert data to a 2D numpy array with 3 columns containing channels X, Y, Z                   
                    segIndPair = segmentByEnergy(accData[:,selectAxis], segIntervalSamp, disp=False)
                    walkDF.append([majorType, subType, accData, accFs, segIndPair])
                    
walkDF = pd.DataFrame(walkDF, columns = ['MajorType','SubType','AccData','AccFs','SegmentIndex'])
display(walkDF)

In [ ]:
'''
Segment the data
'''
def segmentMultichannelData(datain, segmentingInd, disp=False, dispAxis=0, waterfallShift=1000):
    if disp:
        _,ax0 = plt.subplots(1,1,figsize=(14, 20),dpi=72); 
        ax0.set_xlabel('Samples')
        ax0.set_ylabel(row['MajorType'])
        
    dataout = []
    for i in range(len(segmentingInd)-1):
        if(segmentingInd[i+1] - segmentingInd[i]) > minInterval:  # Discard the segment if it is shorter than 'minInterval'
            segmentEndInd = min(segmentingInd[i+1], segmentingInd[i]+maxInterval) # Limit the segment length based on 'maxInterval'
            dataSegment = datain[segmentingInd[i]:segmentEndInd,:] # Get the accelerometer data segment
            if disp:
                ax0.plot(dataSegment[:,-1]+(i*waterfallShift))
            dataout.append(dataSegment)
    return dataout

''' Settings '''
minInterval = 900 # (samples) the minimum interval between two consecutive foot steps
maxInterval = 1300 # (samples) the maximum interval between two consecutive foot steps

datasegDF = [] # Dataframe of the segmented data to be used for classification
for index, row in walkDF.iterrows():
    segmentingInd = row['SegmentIndex'][1] # Use 'energySurgeDownInd' to segment the data
    accSegment = segmentMultichannelData(row['AccData'], segmentingInd, disp=False, dispAxis=2, waterfallShift=1000)
    datasegDF.append([row['MajorType'],row['SubType'],accSegment,row['AccFs']])
datasegDF = pd.DataFrame(datasegDF, columns = ['MajorType','SubType','AccSegment','AccFs'])
display(datasegDF)

In [ ]:
'''
Feature extraction for classification
'''
display(datasegDF['MajorType']) # The index for each class label

selectAxis = 0
dataFeature = []
for index, row in datasegDF.iterrows():
    accSegment = row['AccSegment']
    
    for seg in accSegment:
        
        timeAvg = np.sqrt(np.mean(seg[:,selectAxis]**2)) # Time-averaged energy of ?-axis data
        
        timePeak = np.max(abs(seg[:,selectAxis])) # ?-axis peak
        
        filteredData = highpassFilter(seg[:,selectAxis], cutFreqRatio=0.01, order=8)
        
        spectr, f, _ = plt.magnitude_spectrum(filteredData, Fs=row['AccFs'], window=mlab.window_none);
        centerFreq = np.dot(spectr/np.sum(spectr), f)

        dataFeature.append([timeAvg, timePeak, centerFreq, index])
dataFeature = np.array(dataFeature)

featureNum = dataFeature.shape[1]-1

# fig = plt.figure(figsize=(14, 12))
# ax3D = fig.add_subplot(projection='3d')
_,ax = plt.subplots(featureNum,1,figsize=(14, 8),dpi=72); 
for i in range(8):
    aSurfaceType = dataFeature[dataFeature[:,-1]==i]
    print("%s %s" % (datasegDF.loc[i,'MajorType'], aSurfaceType.shape))
    for j in range(featureNum):
        ax[j].boxplot(aSurfaceType[:,j], positions=[i])
for j in range(featureNum):
    ax[j].set_xticklabels(datasegDF['MajorType'])
#     ax3D.scatter(aSurfaceType[:,0], aSurfaceType[:,1], aSurfaceType[:,2])

In [ ]:
mfccFeature = librosa.feature.mfcc(y=filteredData, sr=row['AccFs'], n_mfcc=4)
print(mfccFeature)

In [ ]:
'''
Classification
'''
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X = dataFeature[:,:-1]
y = dataFeature[:,-1]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=101)

rbf = OneVsRestClassifier(svm.SVC(kernel='rbf', gamma=0.001, C=1)).fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print("Accuracy: %.2f%%, F1: %.2f%%" % (rbf_accuracy*100, rbf_f1*100))

for i in range(8):
    ind = (y_test==i)
    rbf_accuracy = accuracy_score(y_test[ind], rbf_pred[ind])
    rbf_f1 = f1_score(y_test[ind], rbf_pred[ind], average='weighted')
    print("%s (%d samples) - Accuracy: %.2f%%, F1: %.2f%%" % (datasegDF.loc[i,'MajorType'],sum(ind),rbf_accuracy*100, rbf_f1*100))